## Recommender env : design

In [1]:
# HIDDEN
import gym

In [2]:
# TODO: adapt images from Sven for this

#### Simuler le comportement de l'utilisateur

- Simule le comportement de l'utilisateur lorsqu'il répond de façon **répétée** aux recommandations d'articles.
- Le comportement clé à simuler :

&gt; Recommander des articles de "malbouffe" est bon à court terme, mais mauvais à long terme.

- C'est entièrement notre choix en tant que concepteur de l'environnement 
- Tu voudras peut-être simuler/capturer un autre type de comportement d'utilisateur.
  - Ou bien, tire des enseignements des données sur le comportement des utilisateurs - plus d'informations à ce sujet plus tard !
- Mais ceci sera notre exemple courant pour l'instant.

#### Chocolat

- Nous modéliserons chaque article comme ayant un niveau de "douceur" 
- Nous appellerons les articles à forte teneur en sucre des "bonbons".

[](img/candy.jpg)

- Il peut s'agir de vidéos courtes et stupides, de bibelots bon marché en solde, etc.
- Les utilisateurs aiment les bonbons à court terme, mais trop de chocolat entraîne une insatisfaction à long terme.

#### Veggies

- Nous appellerons les articles à faible teneur en sucre des "légumes".

[](img/veggies.jpg)

- Il peut s'agir de documentaires éducatifs, d'articles ennuyeux mais utiles, etc.
- Les utilisateurs n'apprécient pas beaucoup les végés à court terme, mais ils augmentent la satisfaction à long terme.

#### Niveau de sucre

- Nous allons modéliser nos utilisateurs comme ayant un **niveau de sucre** qui mesure la quantité de bonbons qu'ils ont mangés récemment.
- Le taux de sucre de l'utilisateur (ou la notion de taux de sucre) _ne sera pas connu de l'agent_ !
- Mais pour l'instant, nous sommes en train de concevoir le simulateur, donc nous sommes omniscients.

#### Dynamique du niveau de sucre

- Nous devons décider comment le niveau de sucre change avec la consommation de l'article.
- Une approche simple est la suivante :

&gt; Lorsqu'un article est consommé, le niveau de sucre se déplace vers le goût sucré de cet article.

Exemples :

- Si ton taux de sucre est de 0,2 et que tu consommes un article avec un pouvoir sucrant de 0,5, ton taux de sucre augmente ⬆️
- Si ton taux de sucre est de 0,2 et que tu consommes un article ayant un pouvoir sucrant de 0,1, ton taux de sucre diminue ⬇️

#### Dynamique du niveau de sucre

- Comment pouvons-nous représenter cela mathématiquement ?
- Nous pouvons essayer ceci :

&gt; nouveau niveau de sucre = ⍺ (ancien niveau de sucre) + (1 - ⍺) (douceur de l'article)

- Ici, ⍺ est un nombre entre 0 et 1 qui contrôle le degré de "ténacité" du niveau de sucre.

In [3]:
# HIDDEN 
# note the slide above and below are partially the same - just want to hide the bottom half at first

#### Dynamique du niveau de sucre

- Comment pouvons-nous représenter cela mathématiquement ?
- Nous pouvons essayer ceci :

&gt; nouveau niveau de sucre = ⍺ (ancien niveau de sucre) + (1 - ⍺) (douceur de l'article)

- Ici, ⍺ est un nombre entre 0 et 1 qui contrôle le degré de "ténacité" du niveau de sucre.
- Par exemple, si ⍺=1, alors l'équation ci-dessus devient la suivante

&gt; nouveau taux de sucre = ancien taux de sucre

et le taux de sucre ne change jamais. Si ⍺=0, alors nous avons

&gt; nouveau niveau de sucre = douceur de l'article

ce qui signifie qu'un seul article peut complètement changer le niveau de sucre de l'utilisateur.

- Pour les ⍺ entre 0 et 1, nous avons une combinaison de l'ancien niveau de sucre et de la douceur de l'article.

#### Dynamique du niveau de sucre

Nous pouvons mettre en œuvre ce qui précède en utilisant cette fonction :

In [4]:
def update_sugar_level(sugar_level, item_sweetness, alpha=0.9):
    return alpha * sugar_level + (1 - alpha) * item_sweetness

Testons-le pour nous assurer que le comportement a du sens (en utilisant la valeur par défaut de ⍺=0,9) :

In [5]:
sugar_level = 0.2
sugar_level = update_sugar_level(sugar_level, 0.8)
sugar_level

0.26

L'article était sucré (0,8), donc le taux de sucre a beaucoup augmenté.

In [6]:
sugar_level = update_sugar_level(sugar_level, 0.3)
sugar_level

0.264

La douceur de l'article était légèrement supérieure au niveau de sucre, donc le niveau de sucre a légèrement augmenté.

#### Dynamique du niveau de sucre

In [7]:
sugar_level = update_sugar_level(sugar_level, 0.01)
sugar_level

0.2386

L'article n'était pas sucré, donc le taux de sucre a baissé.

#### Effet de l'alpha

Nous pouvons voir qu'avec un alpha plus petit, le niveau de sucre change beaucoup plus rapidement :

In [8]:
sugar_level = update_sugar_level(sugar_level, 0.0, alpha=0.5)
sugar_level

0.1193

#### Récompense

- Ok super, nous avons résolu la question de la dynamique du taux de sucre !
- La deuxième pièce majeure du puzzle est la récompense.
- Ce que nous voulons :

1. Un taux de sucre plus élevé dans l'article entraîne une plus grande récompense (miam, des bonbons !)
2. Un taux de sucre plus élevé entraîne une récompense plus faible (ahh, trop de bonbons !)

Un moyen simple de combiner ces effets est de les multiplier ensemble :

&gt; récompense = douceur de l'article * (1 - niveau de sucre)

#### Mise en œuvre des récompenses

&gt; récompense = douceur de l'article * (1 - niveau de sucre)

Nous pouvons coder cela comme suit :

In [9]:
def reward(sugar_level, item_sweetness):
    return item_sweetness * (1 - sugar_level)

Nous utiliserons ces pièces dans la prochaine section lorsque nous mettrons en place notre environnement !

#### Espace d'observation

- Ensuite, nous devrons mettre en place les observations 
- Nos observations seront les _caractéristiques des articles candidats_.
- Pour simplifier, nous supposerons qu'il n'y a qu'une seule caractéristique, la douceur de l'article.
- Ainsi, l'agent verra un tas de niveaux de douceur et en choisira un.

#### Espace d'action

- Dans cet environnement, l'action est l'élément choisi à recommander, étant donné les candidats.

#### Appliquons ce que nous avons appris !

## Grande vision
<!-- multiple choice -->

Lequel des points suivants est **NON** vrai à propos de l'environnement RL de recommandation simulé que nous créons ?

- [ ] L'environnement contient un modèle très simplifié du comportement des utilisateurs, mais un agent entraîné peut tout de même être utile pour faire des recommandations.
- [ ] L'environnement représente avec précision le comportement des vrais utilisateurs.
- [ ] L'environnement est un bon point de départ et nous voudrons peut-être ajouter de la complexité au fur et à mesure que notre travail avance.
- [ ] L'environnement capture la notion selon laquelle les utilisateurs réagiront différemment à différents éléments, et cette réaction peut dépendre de leur historique.

## Recommander des récompenses
<!-- multiple choice -->

Rappelle-toi que notre fonction de récompense est

&gt; récompense = douceur de l'article * (1 - niveau de sucre)

#### Satisfaction à court terme

Vrai ou faux : à tout moment, la récompense _immédiate_ est _toujours_ plus grande pour les bonbons que pour les légumes.

- [ ] Vrai | La récompense immédiate est directement proportionnelle à la douceur de l'article.
- [ ] Faux | Regarde de plus près la formule ci-dessus !

#### Satisfaction à long terme

Vrai ou faux : à tout moment, la récompense _totale à long terme_ est _toujours_ plus grande pour recommander des légumes que pour des bonbons.

- [ ] Vrai | Il est compliqué de déterminer ce qui sera le mieux à long terme - c'est ce que notre agent doit apprendre !
- [x] Faux

## Accident de sucre
<!-- coding exercise -->

Supposons que ton taux de sucre commence à 0,5 et qu'à chaque étape, tu n'as le choix qu'entre deux aliments, le méga-légume (goût sucré = 0) et le méga-bonbon (goût sucré = 1). Tu feras 3 recommandations à la suite, en utilisant alpha = 0,7. Utilise la fenêtre de codage ci-dessous pour jouer avec différentes options et trouver la meilleure séquence de recommandations en termes de récompense _totale_ 

In [10]:
# EXERCISE

def update_sugar_level(sugar_level, item_sweetness, alpha=0.9):
    return alpha * sugar_level + (1 - alpha) * item_sweetness

def reward(sugar_level, item_sweetness):
    return item_sweetness * (1 - sugar_level)

# MODIFY THIS LIST
# But make sure it always contains 3 items, each 0 or 1
recommendations = [0, 0, 0]

# starting sugar level
sugar_level = 0.5

total_reward = 0

for item_sweetness in recommendations:
    
    # add reward
    immediate_reward = reward(sugar_level, item_sweetness)
    total_reward += immediate_reward
    
    # update sugar level
    sugar_level = update_sugar_level(sugar_level, item_sweetness, alpha=0.7)
    
    print(f"  Received reward {immediate_reward:.5f}, new sugar level {sugar_level:.5f}")
    
print("Total reward after 5 recommendations:", total_reward)

  Received reward 0.00000, new sugar level 0.35000
  Received reward 0.00000, new sugar level 0.24500
  Received reward 0.00000, new sugar level 0.17150
Total reward after 5 recommendations: 0.0


In [11]:
# SOLUTION

def update_sugar_level(sugar_level, item_sweetness, alpha=0.9):
    return alpha * sugar_level + (1 - alpha) * item_sweetness

def reward(sugar_level, item_sweetness):
    return item_sweetness * (1 - sugar_level)

# MODIFY THIS LIST
# But make sure it always contains 3 items, each 0 or 1
recommendations = [0,1,1]

# starting sugar level
sugar_level = 0.5

total_reward = 0

for item_sweetness in recommendations:
    
    # add reward
    immediate_reward = reward(sugar_level, item_sweetness)
    total_reward += immediate_reward
    
    # update sugar level
    sugar_level = update_sugar_level(sugar_level, item_sweetness, alpha=0.7)
    
    print(f"  Received reward {immediate_reward:.5f}, new sugar level {sugar_level:.5f}")
    
print("Total reward after 5 recommendations", total_reward)

  Received reward 0.00000, new sugar level 0.35000
  Received reward 0.65000, new sugar level 0.54500
  Received reward 0.45500, new sugar level 0.68150
Total reward after 5 recommendations 1.105


#### Quelle était la meilleure stratégie dans cet exemple ?

- [ x ] 1 légume pour faire baisser le taux de sucre, puis 2 bonbons pour cette douce récompense.
- [ ] Des bonbons, puis des légumes pour être en bonne santé, puis encore des bonbons.
- [ ] Légumes à volonté !
- [ ] Bonbons à volonté !